# Como disponibilizar seu modelo na internet

Nesse passo, iremos utilizar o Django, um _framework_ web robusto escrito em Python, para fazer o _deploy_ do seu modelo. Dessa forma, qualquer um poderá acessar sua aplicação e testá-la.

## Um pouco sobre Django...

Django é um framework de alto nível, escrito em Python que encoraja o desenvolvimento limpo de aplicações web.

Desenvolvido por experientes desenvolvedores, Django toma conta da parte pesada do desenvolvimento web, como: 
- Tratamento de requisições
- Mapeamento objeto-relacional
- Preparação de respostas HTTP
- Autenticação
- Autorização; e muito mais

Dessa forma, você gasta seu esforço com aquilo que realmente interessa: suas **regras de negócio**!

Foi desenvolvido com uma preocupação extra em segurança, evitando os mais comuns ataques, como _Cross Site Scripting_ (XSS), _Cross Site Request Forgery_ (CSRF), _SQL injection_, entre outros.

É bastante **escalável**: Django foi desenvolvido para tirar vantagem da maior quantidade de hardware possível (desde que você queira). Django usa uma arquitetura “zero-compartilhamento”, o que significa que você pode adicionar mais recursos em qualquer nível: servidores de banco de dados, cache e/ou servidores de aplicação.

### Sua arquitetura

Diferente da arquitetura de outros _frameworks_ web que seguem o padrão MVC (_Model-View-Controller_), o Django possui uma arquitetura sutilmente diferente: seus desenvolvedores o denominaram de **MTV** ou _**Model-Template-View**_.

As responsabilidades de cada camada são:
- _**Model**_: Faz o mapeamento dos objetos da sua aplicação com tabelas relacionais e gerencia as transações que acessam o banco de dados.
- _**Template**_: Contém a camada de apresentação da sua aplicação (arquivos HTML, arquivos de estilo CSS, possíveis bibliotecas Javascript e etc). É a **cara** do seu projeto.
- _**View**_: Essa camada tem a responsabilidade de processar as requisições vindas dos usuários, formular uma resposta e enviá-la. É aqui que residem nossas **lógicas de negócio**.

### Fluxo de uma requisição

O fluxo de uma Requisição HTTP dentro do Django pode ser demonstrado da seguinte forma:

![Fluxo de uma requisição](https://raw.githubusercontent.com/viniciusramos91/ai-web-app/master/notebook/django-architecture.png)
_Fonte: [Python Academy](https://pythonacademy.com.br/blog/desenvolvimento-web-com-python-e-django-introducao)_

Agora que você já sabe (quase) tudo sobre o Django, vamos para a **prática**

### Primeiros passos

Se você estiver utilizando a Google Cloud Platform, abra uma nova janela de terminal (_Open in browser window_):

![Google Cloud Platform](https://raw.githubusercontent.com/viniciusramos91/ai-web-app/master/notebook/gcp.png)

Agora, clone o projeto Django:

In [ ]:
# Vá para pasta home
%cd ~

# Clone o projeto
!git clone https://github.com/viniciusramos91/ai-web-app.git
    
# Vá para dentro do projeto
%cd ./ai-web-app

Agora, instale as dependências do projeto (no caso apenas o Django), executando a célula abaixo:

In [ ]:
!pip install -r requirements.txt --user

A estrutura do projeto é:

```
requirements.txt
manage.py
webapp/
    > settings.py
    > urls.py
    > wsgi.py
```

Explicando cada arquivo:

- `requirements.txt`: Arquivo de dependências de projetos Python.
- `manage.py`: Arquivo gerado automaticamente pelo Django que expõe comandos importantes para manutenção da nossa aplicação.
- `webapp/settings.py`: Arquivo muito importante com as configurações do nosso projeto, como configurações do banco de dados, aplicativos instalados, configuração de arquivos estáticos e muito mais.
- `webapp/urls.py`: Nossa URLConf - aqui vamos dizer ao Django quem responde a qual URL.
- `webapp/wsgi.py``: Aqui configuramos a interface entre o servidor de aplicação e nossa aplicação Django.

Com a estrutura acima criada, é necessário fazer a carga inicial do banco de dados do Django. 

Para isso, executaremos o comando `migrate` do Django conforme a célula abaixo (deverá ser criado o banco de dados `db.sqlite3` na raíz do projeto).

In [ ]:
!python manage.py migrate

Agora, execute o comando abaixo, abra seu browser e acesse sua instância na porta 8000. 

Para **parar** sua execução, clique no botão de _Stop_ (barra de ferramentas do Jupyter - ao lado do _Run_).

In [ ]:
!python manage.py runserver 0.0.0.0:8000

A tela inicial do projeto deverá ser mostrada:

![Django default](https://raw.githubusercontent.com/viniciusramos91/ai-web-app/master/notebook/index.png)

E ao fazer o upload de uma imagem, a seguinte tela será exibida:

![Resultado](https://raw.githubusercontent.com/viniciusramos91/ai-web-app/master/notebook/result.png)

**Obs**: Ainda não estárá funcionando pois seu modelo não foi configurado.

### Entendendo um pouco do projeto Django

Com o objetivo de criar uma página gerenciada pelo Django, é necessário fazermos, basicamente, três passos:
- Criar a página html que será mostrada ao usuário.
- Escrever a lógica no arquivo `views.py` (ainda não criado).
- Adicionar a rota no arquivo `urls.py` .

### Arquivo `index.html`

O arquivo `webapp/templates/webapp/index.html` contém o código HTML da nossa página inicial. 

Ela é responsável por fazer o _upload_ de uma foto ao Django.

Sem muito mistério aqui...

### Arquivo `views.py`

No Django, o código responsável por processar a requisição do usuário e retornar uma resposta reside em uma `View`.

No nosso caso, nossa `View`:
- Recebe uma imagem
- Trata a imagem
- Carrega nosso modelo
- Faz o `predict` utilizando a imagem tratada
- Retorna a classe daquela imagem

Para que seu modelo seja executado, altere o arquivo `webapp/views.py`, onde se encontra o seguinte trecho de código:

```python
learn = load_learner('/home/jupyter/playground', fname='modelo.pkl')
```

_(Aponte para o arquivo do seu modelo)_

O código da nossa `View` é o seguinte (siga os comentários):

```python
from django.shortcuts import render
from fastai import *
from fastai.vision import *
from django.http import HttpRequest, HttpResponse
from io import BytesIO

# View da página principal
# =======================================================================

def index(request: HttpRequest) -> HttpResponse:

    # Verifica o método HTTP
    if request.method == 'GET':
        # Se GET, renderiza a página HTML
        return render(request, 'webapp/index.html', status=200)
    
    # Se POST
    elif request.method == 'POST':
        # Pega a image da requisição
        image = request.FILES['image']

        # Lê os bytes da imagem
        data = BytesIO(image.read())

        # Chama o PIL para converter a imagem em preto e branco
        img = PIL.Image.open(data).convert('LA')

        # Salva a imagem em disco
        img.save('imagem.png')

        # Abre a imagem com o fast.ai
        img = open_image('imagem.png')

        # Carrega o conjunto padrão de transformações do fast.ai para aplicar na imagem
        tfms = get_transforms()

        # Aplica as transofmrações na imagem
        for transformation in tfms:
            fastai_img = img.apply_tfms(transformation, size=224)

        # Carrega o modelo
        learn = load_learner('/home/jupyter/playground', fname='modelo.pkl')

        # Executa o modelo sobre a imagem
        pred_class, pred_idx, outputs = learn.predict(fastai_img)

        return HttpResponse(pred_class, status=200)

    else:
        return HttpResponse('Método não permitido', status=405)
```     

### Arquivo `urls.py`

Para que o Django chame nossa `View` é necessário que sua rota esteja configurada corretamente.

Isso é feito no arquivo `webapp/urls.py`. Veja como é simples (siga os comentários):

```python
from django.contrib import admin
from django.urls import path
from webapp.views import index

# Lista contendo todos os caminhos da sua aplicação
urlpatterns = [
    # Sim, o Django já vem com uma área de administrador por padrão!
    # Acesse: https://docs.djangoproject.com/en/2.0/intro/tutorial02/#introducing-the-django-admin
    path('admin/', admin.site.urls),
    
    # Rota que encaminha as requisições do caminho raíz para nossa view index()
    path('', index)
]
```